In [2]:
import pandas as pd
import numpy as np

In [3]:
ind_games = pd.read_csv("data/final_data/all_games_16_18.csv")

In [4]:
ind_games = ind_games.drop('Team2_int_re_number',axis=1)

In [5]:
clean_ind_games = ind_games.drop(['game_id','Team1_team_id','Team2_team_id','Team1','Team2'],axis=1)

### Target split

In [6]:
clean_ind_games['Team2_win'].value_counts()/clean_ind_games.shape[0]

1    0.622561
0    0.377439
Name: Team2_win, dtype: float64

### Re-shuffle dataframe to get equal target variables 

In [7]:
target_yes = clean_ind_games[clean_ind_games['Team2_win'] == 1]
target_no = clean_ind_games[clean_ind_games['Team2_win']==0]

In [8]:
sample_no = int((target_yes.shape[0] - target_no.shape[0])/2)

In [9]:
flip_df = target_yes.sample(n=sample_no)
remove_index = flip_df.index.tolist()

In [10]:
flip_df.head()

,Team1_team_score,Team2_team_score,Team1_first_downs,Team1_passing_first_downs,Team1_rushing_first_downs,Team1_rushing_yds,Team1_rushing_attempts,Team1_passing_attempts,Team1_passing_completions,Team1_passing_interceptions,...,Team2_third_down_suc,Team2_fourth_down_attempts,Team2_fourth_down_suc,Team1_third_down_conv_pct,Team2_third_down_conv_pct,Team1_fourth_down_conv_pct,Team2_fourth_down_conv_pct,Team1_turnover_margin,Team2_turnover_margin,Team2_win
544,16,48,16,6,9,97,29,41,18,1,...,8,0,0,0.266667,0.571429,1.0,0.000000,-1,1,1
631,10,48,17,7,8,122,21,41,20,1,...,6,3,3,0.562500,0.500000,0.0,1.000000,1,-1,1
1372,17,51,24,11,9,197,38,29,13,0,...,9,0,0,0.272727,0.600000,0.0,0.000000,1,-1,1
127,36,38,22,5,15,118,37,32,21,0,...,6,2,2,0.500000,0.500000,1.0,1.000000,-1,1,1
68,23,27,22,3,15,77,23,44,29,3,...,4,3,2,0.500000,0.285714,0.0,0.666667,3,-3,1


In [11]:
orig_cols = flip_df.columns.tolist()

In [12]:
target_yes = target_yes.drop(remove_index)

In [13]:
### Rename and flip values
t1_cols_rename = flip_df[flip_df.columns[flip_df.columns.str.contains('Team1')]].columns.tolist()
sep_df = flip_df[t1_cols_rename]
t1_cols_rename = [word.replace('1','2') for word in t1_cols_rename]
sep_df.columns = t1_cols_rename

t2_cols_rename = flip_df[flip_df.columns[flip_df.columns.str.contains('Team2')]].columns.tolist()
sep_df2 = flip_df[t2_cols_rename]
t2_cols_rename = [word.replace('2','1') for word in t2_cols_rename]
sep_df2.columns = t2_cols_rename

In [14]:
flip_df = pd.concat([sep_df,sep_df2],axis=1)
flip_df = flip_df.drop('Team1_win',axis=1)
flip_df['Team2_win'] = np.where(flip_df['Team2_team_score']>flip_df['Team1_team_score'],1,0)
flip_df = flip_df[orig_cols]

In [15]:
target_yes.shape

(1179, 63)

In [16]:
target_no = pd.concat([target_no,flip_df],axis=0)

In [17]:
clean_ind_games2 = pd.concat([target_yes,target_no],axis=0).sample(frac=1).reset_index(drop=True)

In [18]:
clean_ind_games2.to_csv('data/final_data/clean_shuffled_games.csv',index=False)

### Create Scale Model 

In [19]:
scale_df = clean_ind_games2.drop(['Team2_win','Team1_team_score','Team2_team_score'],axis=1).convert_objects(convert_numeric=True)
scale_df = scale_df.fillna(0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(scale_df)

In [21]:
from sklearn.externals import joblib 
scaler_filename = "models/scaler.sav"
joblib.dump(scaler, scaler_filename) 

['models/scaler.sav']

### Train/Test

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
clean_ind_games2 = clean_ind_games2.drop(['Team1_team_score','Team2_team_score'],axis=1).convert_objects(convert_numeric=True)
clean_ind_games2 = clean_ind_games2.fillna(0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [24]:
trainingSet, testSet = train_test_split(clean_ind_games2, test_size=0.3,stratify=None)

In [25]:
trainingSet.to_csv("data/final_data/clean_train.csv",index=False)
testSet.to_csv("data/final_data/clean_test.csv",index=False)

### Prepare Data to search through

In [32]:
search_df = ind_games.drop(['Team1_team_id','Team2_team_id','Team1_team_score','Team2_team_score',
                           'Team2_win'],axis=1)

In [34]:
game_ids = search_df['game_id']

In [44]:
team1_df = search_df[search_df.columns[search_df.columns.str.contains('Team1')].tolist()]
team2_df = search_df[search_df.columns[search_df.columns.str.contains('Team2')].tolist()]

In [47]:
t1_cols_rename = team2_df.columns
t1_cols_rename = [word.replace('2','1') for word in t1_cols_rename]
team2_df.columns = t1_cols_rename

In [53]:
search_df_fin = pd.concat([pd.concat([game_ids,team1_df],axis=1),pd.concat([game_ids,team2_df],axis=1)],axis=0)

In [57]:
search_df_fin.to_csv('data/final_data/team_games_data.csv',index=False)

In [60]:
search_df_fin[search_df_fin['game_id'] == 'west-virginia-texas-tech']

,game_id,Team1,Team1_first_downs,Team1_passing_first_downs,Team1_rushing_first_downs,Team1_rushing_yds,Team1_rushing_attempts,Team1_passing_attempts,Team1_passing_completions,Team1_passing_interceptions,...,Team1_kick_re_number,Team1_kick_re_yds,Team1_int_re_yds,Team1_third_down_attempts,Team1_third_down_suc,Team1_fourth_down_attempts,Team1_fourth_down_suc,Team1_third_down_conv_pct,Team1_fourth_down_conv_pct,Team1_turnover_margin
0,west-virginia-texas-tech,West Virginia,26,8,15,119,32,41,27,0,...,0,0,106,14,7,1,0,0.500000,0.000000,-3
1561,west-virginia-texas-tech,West Virginia,28,15,12,332,46,31,21,0,...,1,25,0,13,7,1,1,0.538462,1.000000,-1
0,west-virginia-texas-tech,Texas Tech,28,13,13,168,42,47,25,3,...,0,0,0,17,7,3,3,0.411765,1.000000,3
1561,west-virginia-texas-tech,Texas Tech,21,6,12,34,27,50,32,1,...,1,25,0,19,10,3,1,0.526316,0.333333,1
